In [7]:
import uproot
from pathlib import Path
from typing import Union
import logging
import numpy as np
from matplotlib import pyplot as plt
import hist

import mplhep as hep

hep.style.use(hep.style.ATLAS)

from utils.histograms import (
    TH1tohist,
    histtoTH1,
    RebinTH1,
    Rebinhist,
)
from utils.constants import myoutput_path
from utils.processing import (
    logging_setup,
    check_inputpath,
    check_outputpath,
)

In [2]:
histograms1 = uproot.open("/data/hrzhao/Samples/Run2_WZ/Finebin/ExportHistograms.root")
histograms2 = uproot.open(
    "/data/hrzhao/Samples/ssWWWZ_run3/information_Run2_WZHp/fit_inputs/histograms/Final/regularGMANN.root"
)

In [16]:
hist_identifiers = ["WZEW", "WZQCD", "VVV", "ZZ", "data"]
# "FakeRenorm" is gone in regularGMANN.root
region_labels = ["GMMVASR", "GMMVACR", "ZZCRJJ"]
m_WZ_bin_edges = np.array([150, 200, 230, 270, 310, 350, 390, 480, 660, 1200])
m_WZ_bin_widths = np.diff(m_WZ_bin_edges)
sig_mass_WZ = [200, 300, 350, 500]
# sig_mass_WZ = [
#     200,
#     225,
#     250,
#     275,
#     300,
#     325,
#     350,
#     375,
#     400,
#     425,
#     450,
#     475,
#     500,
#     525,
#     550,
#     600,
#     700,
#     800,
#     900,
#     1000,
# ]

In [4]:
output_path = check_outputpath("comparision_plots")

In [19]:
bkg_hists = {}
for region_label in region_labels:
    for hist_identifier in hist_identifiers:
        hist_name = f"{hist_identifier}_M_WZ_{region_label}"  # WZEW_M_WZ_GMMVASR
        unbined_hist = histograms1[hist_name].to_hist()
        rebined_hist = Rebinhist(unbined_hist, m_WZ_bin_edges)

        # Merge the overflow bin to the last bin
        rebined_hist[-1] += rebined_hist[hist.overflow]
        rebined_hist[0] += rebined_hist[hist.underflow]
        rebined_hist[hist.overflow] = (0.0, 0.0)
        rebined_hist[hist.underflow] = (0.0, 0.0)
        hist1 = rebined_hist.copy()

        _hist2 = histograms2[hist_name].to_hist()
        hist2 = hist.Hist(
            hist.axis.Variable(m_WZ_bin_edges, flow=True),
            storage=hist.storage.Weight(),
        )
        hist2[:] = _hist2.view()

        fig = plt.figure()
        main_ax_artists, sublot_ax_arists = hist1.plot_ratio(
            hist2,
            rp_ylabel=r"Ratio",
            rp_num_label="FineBin",
            rp_denom_label="regularGMANN",
            rp_uncert_draw_type="bar",  # line or bar
        )
        plt.suptitle(hist_name)
        fig.savefig(output_path / f"{hist_name}.png")

        plt.close()

        bkg_hists[hist_name] = [hist1, hist2]

Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warn

In [18]:
# for signal
sig_output_path = check_outputpath("comparision_plots/signal")

for region_label in region_labels:
    for wp_mass in sig_mass_WZ:
        hist_identifier = f"GMvbf{wp_mass}"
        hist_name = f"{hist_identifier}_M_WZ_{region_label}"  # WZEW_M_WZ_GMMVASR
        unbined_hist = histograms1[hist_name].to_hist()
        rebined_hist = Rebinhist(unbined_hist, m_WZ_bin_edges)

        # Merge the overflow bin to the last bin
        rebined_hist[-1] += rebined_hist[hist.overflow]
        rebined_hist[0] += rebined_hist[hist.underflow]
        rebined_hist[hist.overflow] = (0.0, 0.0)
        rebined_hist[hist.underflow] = (0.0, 0.0)
        hist1 = rebined_hist.copy()

        _hist2 = histograms2[hist_name].to_hist()
        hist2 = hist.Hist(
            hist.axis.Variable(m_WZ_bin_edges, flow=True),
            storage=hist.storage.Weight(),
        )
        hist2[:] = _hist2.view()

        try:
            fig = plt.figure()
            main_ax_artists, sublot_ax_arists = hist1.plot_ratio(
                hist2,
                rp_ylabel=r"Ratio",
                rp_num_label="FineBin",
                rp_denom_label="regularGMANN",
                rp_uncert_draw_type="bar",  # line or bar
            )
            plt.suptitle(hist_name)
            fig.savefig(sig_output_path / f"{hist_name}.png")
        except:
            print(f"Error in plotting {hist_name}")
        plt.close()

Error in plotting GMvbf300_M_WZ_GMMVASR


/data/hrzhao/sw/miniconda3/envs/ssWWWZjj/lib/python3.10/site-packages/mplhep/error_estimation.py:49: RuntimeWarning: divide by zero encountered in divide
  counts = sumw / scale


Error in plotting GMvbf500_M_WZ_GMMVASR
Error in plotting GMvbf350_M_WZ_GMMVACR


/data/hrzhao/sw/miniconda3/envs/ssWWWZjj/lib/python3.10/site-packages/mplhep/error_estimation.py:49: RuntimeWarning: divide by zero encountered in divide
  counts = sumw / scale
/data/hrzhao/sw/miniconda3/envs/ssWWWZjj/lib/python3.10/site-packages/mplhep/error_estimation.py:49: RuntimeWarning: divide by zero encountered in divide
  counts = sumw / scale
/data/hrzhao/sw/miniconda3/envs/ssWWWZjj/lib/python3.10/site-packages/mplhep/error_estimation.py:49: RuntimeWarning: divide by zero encountered in divide
  counts = sumw / scale


Error in plotting GMvbf300_M_WZ_ZZCRJJ
Error in plotting GMvbf350_M_WZ_ZZCRJJ
Error in plotting GMvbf500_M_WZ_ZZCRJJ


Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: xaxis (Potential memory leak).


# Count Yield

In [26]:
rows = [*bkg_hists.keys()][:5]

In [33]:
dict_yield = {}
for row in rows:
    dict_yield[row] = [_hist.values().sum() for _hist in bkg_hists[row]]

In [34]:
dict_yield

{'WZEW_M_WZ_GMMVASR': [25.075572729110718, 25.250463128089905],
 'WZQCD_M_WZ_GMMVASR': [36.52554890885949, 36.81976318359375],
 'VVV_M_WZ_GMMVASR': [0.8184943427331746, 0.17784502170979977],
 'ZZ_M_WZ_GMMVASR': [4.559074188102386, 4.591837331652641],
 'data_M_WZ_GMMVASR': [66.0, 66.0]}

In [38]:
import pandas as pd

pd.DataFrame.from_dict(dict_yield, orient="index", columns=["FineBin", "regularGMANN"])

,FineBin,regularGMANN
WZEW_M_WZ_GMMVASR,25.075573,25.250463
WZQCD_M_WZ_GMMVASR,36.525549,36.819763
VVV_M_WZ_GMMVASR,0.818494,0.177845
ZZ_M_WZ_GMMVASR,4.559074,4.591837
data_M_WZ_GMMVASR,66.000000,66.000000
